# Test macro for Bokeh client visualization
* it is aimed to be used for develoopers to test feature of bokeh interactive  - standalone plotting

In [ ]:
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.layouts import column, row
from bokeh.models import CustomJS, ColumnDataSource, Slider
from RootInteractive.InteractiveDrawing.bokeh.bokehDraw import *
output_notebook()

In [ ]:
#init test random data
df = pd.DataFrame(np.random.randint(0, 100, size=(100, 4)), columns=list('ABCD'))
df.head(10)
df.metaData = {'A.AxisTitle': "A (cm)", 'B.AxisTitle': "B (cm/s)", 'C.AxisTitle': "C (s)", 'D.AxisTitle': "D (a.u.)"}
print(df.metaData)

In [ ]:
bokehDrawArray?

In [ ]:
figureArray = [
        [['A'], ['D+A','C-A'], {"size": 1}],
        [['A'], ['C+A', 'C-A']],
    ]
figureLayout: str = '((0),(1, x_visible=1),commonX=1,x_visible=1,y_visible=0,plot_height=250,plot_width=1000)'
tooltips = [("VarA", "(@A)"), ("VarB", "(@B)"), ("VarC", "(@C)"), ("VarD", "(@D)")]
pAll,layoutList,source =bokehDrawArray(df, "A>0", figureArray, layout=figureLayout, color="blue", size=4, tooltips=tooltips)
show(pAll)

In [ ]:
logging.disable(1)

In [ ]:
def makeJScallback(widgetDict):
    """
    """
    code="\
        var dataOrig = cdsOrig.data;  \
        var dataSel  = cdsSel.data;  "
    for iW in widgetDict:
        print(iW)
        #print(type(iW).__name__)
    callback = CustomJS(args=widgetDict, code=code)
    return callback

def test_FigureArray():
    """
    Test of the bokehDrawArray to draw figure array -
    :return:
    """
    # 1.) create figures
    figureArray = [
        [['A'], ['D+A','C-A'], {"size": 8}],
        [['A'], ['C+A', 'C-A']],
        [['A'], ['sin(A/10)', 'sin(A/20)*0.5', 'sin(A/40)*0.25'], {"size": 10}],
    ]
    figureLayout: str = '((0,1, plot_height=200),(2, x_visible=1),commonX=1,x_visible=1,y_visible=0,plot_height=150,plot_width=1000)'
    tooltips = [("VarA", "(@A)"), ("VarB", "(@B)"), ("VarC", "(@C)"), ("VarD", "(@D)")]
    dummy,source,figureList  = bokehDrawArray(df, "A>0", figureArray, layout=figureLayout, color="blue", size=4, tooltips=tooltips)
    # 2.) make CDS
    cdsOrig = source
    cdsSel = copy.copy(source)
    widgetDict={"cdsOrig":cdsOrig, "cdsSel":cdsSel}
    sliders=[]
    # 3.) create sliders 
    for a in df.columns:
        length=df[a].max()-df[a].min()
        sliderRange= RangeSlider(start=df[a].min(), end=df[a].max(), value=(df[a].min(), df[a].max()), step=length*0.01, title=a+".Range")
        sliders.append([sliderRange])
        widgetDict[a+".Range"]=sliderRange
    # make callback
    callback=makeJScallback(widgetDict)
    for a in sliders:
        figureList.append(a)    
    show(gridplotRow(figureList))
    #print(widgetDict)

In [ ]:
test_FigureArray()